In [ ]:
import os

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Set path

In [ ]:
GOOGLE_DRIVE_PATH_AFTER_MYDRIVE = 'CW_Folder_PG'
GOOGLE_DRIVE_PATH = os.path.join('drive', 'My Drive', GOOGLE_DRIVE_PATH_AFTER_MYDRIVE)
print(os.listdir(GOOGLE_DRIVE_PATH))

['.DS_Store', 'CW_Dataset', 'Models', 'Code', 'Video']


# Load images and labels


The train and test datasets were saved in pkl files (in the SVM ipynb file) for faster loading

In [ ]:
import pickle

# set paths where the pkl files are located
train_data_path = os.path.join(GOOGLE_DRIVE_PATH, 'CW_Dataset/train_data.pkl')
test_data_path = os.path.join(GOOGLE_DRIVE_PATH, 'CW_Dataset/test_data.pkl')

# load train data
with open(train_data_path, 'rb') as f:
    train_images, train_labels = pickle.load(f)

# load test data
with open(test_data_path, 'rb') as f:
    test_images, test_labels = pickle.load(f)

# Data preprocessing

Train and validation set

In [ ]:
import pickle
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import img_to_array, array_to_img
from tensorflow.keras.utils import to_categorical
import cv2
import tensorflow as tf


# set new size
new_size = (224, 224)

#resize images with interpolation
#reference: https://stackoverflow.com/questions/23853632/which-kind-of-interpolation-best-for-resizing-image
train_images_resized = [cv2.resize(image, new_size, interpolation=cv2.INTER_LANCZOS4) for image in train_images]

# convert resized images to numpy arrays
train_images_normalized = np.array(train_images_resized)

# normalize pixels
train_images_normalized = train_images_normalized.astype('float32') / 255.0

# convert to one hot endcoded vectors
train_labels = to_categorical(train_labels, 3) # number of classes is 3

# split training data into 80% training and 20% validation
train_images, val_images, train_labels, val_labels = train_test_split(train_images_normalized, train_labels, test_size=0.2, random_state=42)

# set batch size
batch_size = 32

# create tensorflow dataset with train images and train labels
train_dataset = tf.data.Dataset.from_tensor_slices((train_images, train_labels)).shuffle(len(train_images)).batch(batch_size)

# create tensorflow dataset with validation images and validation labels
val_dataset = tf.data.Dataset.from_tensor_slices((val_images, val_labels)).batch(batch_size)



Test set

In [ ]:
# resize test images, new_size defined in previous cell
# #reference: https://stackoverflow.com/questions/23853632/which-kind-of-interpolation-best-for-resizing-image
test_images_resized = [cv2.resize(image, new_size, interpolation=cv2.INTER_LANCZOS4) for image in test_images]

# convert to numpy array
test_images_normalized = np.array(test_images_resized)

# normalize pixels
test_images_normalized = test_images_normalized.astype('float32') / 255.0


# convert to one hot encoded vectors
test_labels = to_categorical(test_labels, 3) # 3 classes

# set batch size
batch_size = 32

# create tensorflow dataset with test images and test labels
test_dataset = tf.data.Dataset.from_tensor_slices((test_images_normalized, test_labels)).batch(batch_size)

# Define train and validate function

In [ ]:
from sklearn.metrics import classification_report

# create train and evaluate function
# params: takes im model, train dataset, validation dataset, and number of epochs
def train_and_validate(model, train_dataset, val_dataset, epochs):
    # early stopping if no improvement in 25 epochs
    early_stopping = EarlyStopping(monitor='val_loss', patience=25)

    # train model with train data
    model.fit(train_dataset, epochs=epochs, validation_data=val_dataset, callbacks=[early_stopping])

    # evaluate model with validation data
    val_loss, val_accuracy = model.evaluate(val_dataset)
    print(f"Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}")

    # make predictions with validation data
    val_images, val_labels = next(iter(val_dataset))
    y_pred = np.argmax(model.predict(val_images), axis=1)

    # create and print classification report
    print("\nClassification Report:")
    print(classification_report(np.argmax(val_labels, axis=1), y_pred))

# set number of epochs
epochs = 250

# Models

## Model 1

No data balancing

### Define model

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Dropout, Flatten, Dense, BatchNormalization
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
import numpy as np

# reference: https://www.kaggle.com/code/gulgaishatemerbekova/face-mask-detection-cnn-resnet50

# define input shape, takes 224x224 images with 3 channels
input_shape = (224, 224, 3)
input_data = Input(shape=input_shape)

# conv layers
x = Conv2D(32, (3, 3), activation='relu')(input_data)
x = BatchNormalization()(x)
x = MaxPooling2D(pool_size=(2, 2))(x)
x = Dropout(0.25)(x)

x = Conv2D(64, (3, 3), activation='relu')(x)
x = BatchNormalization()(x)
x = MaxPooling2D(pool_size=(2, 2))(x)
x = Dropout(0.25)(x)

x = Conv2D(128, (3, 3), activation='relu')(x)
x = BatchNormalization()(x)
x = MaxPooling2D(pool_size=(2, 2))(x)
x = Dropout(0.25)(x)

# fully connected layers
x = Flatten()(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x),
output = Dense(3, activation='softmax')(x) # apply softmax for multiclassification

# create model
cnn2 = Model(inputs=input_data, outputs=output)

# compile model with adam optimizer and categorical cross entropy loss function
cnn2.compile(optimizer='adam', loss=CategoricalCrossentropy(), metrics=['accuracy'])

# get model summary
cnn2.summary()


Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 conv2d_6 (Conv2D)           (None, 222, 222, 32)      896       
                                                                 
 batch_normalization_3 (Bat  (None, 222, 222, 32)      128       
 chNormalization)                                                
                                                                 
 max_pooling2d_6 (MaxPoolin  (None, 111, 111, 32)      0         
 g2D)                                                            
                                                                 
 dropout_7 (Dropout)         (None, 111, 111, 32)      0         
                                                                 
 conv2d_7 (Conv2D)           (None, 109, 109, 64)      1849

### Train model

In [ ]:
train_and_validate(cnn2, train_dataset, val_dataset, epochs)

Epoch 1/250
60/60 [==============================] - 112s 2s/step - loss: 15.2232 - accuracy: 0.8658 - val_loss: 354.8661 - val_accuracy: 0.1294
Epoch 2/250
60/60 [==============================] - 108s 2s/step - loss: 3.8505 - accuracy: 0.9097 - val_loss: 113.8887 - val_accuracy: 0.1336
Epoch 3/250
60/60 [==============================] - 108s 2s/step - loss: 2.7856 - accuracy: 0.9253 - val_loss: 55.5404 - val_accuracy: 0.3779
Epoch 4/250
60/60 [==============================] - 114s 2s/step - loss: 1.8215 - accuracy: 0.9227 - val_loss: 17.9407 - val_accuracy: 0.7954
Epoch 5/250
60/60 [==============================] - 108s 2s/step - loss: 1.4765 - accuracy: 0.9295 - val_loss: 30.6446 - val_accuracy: 0.8017
Epoch 6/250
60/60 [==============================] - 108s 2s/step - loss: 1.5222 - accuracy: 0.9269 - val_loss: 19.6332 - val_accuracy: 0.7077
Epoch 7/250
60/60 [==============================] - 107s 2s/step - loss: 0.9409 - accuracy: 0.9279 - val_loss: 20.5256 - val_accuracy: 0.8

In [ ]:
cnn2.save(os.path.join(GOOGLE_DRIVE_PATH, 'Models/custom_cnn2.keras'))

### Test model

In [ ]:
test_loss, test_accuracy = cnn2.evaluate(test_dataset)
print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}")

15/15 [==============================] - 6s 409ms/step - loss: 19.6849 - accuracy: 0.8908
Test Loss: 19.6849, Test Accuracy: 0.8908


In [ ]:
from sklearn.metrics import classification_report
import numpy as np

# make predictions on test dataset
predictions = cnn2.predict(test_dataset)

# convert to class labels
predicted_classes = np.argmax(predictions, axis=1)

# convert the one-hot encoded true labels to class indices
true_labels = []
for _, label in test_dataset:
    true_labels.extend(np.argmax(label.numpy(), axis=1))

# get classification report
report = classification_report(true_labels, predicted_classes)

# print report
print(report)

15/15 [==============================] - 6s 418ms/step
              precision    recall  f1-score   support

           0       0.86      0.59      0.70        51
           1       0.92      0.97      0.94       388
           2       0.08      0.05      0.06        19

    accuracy                           0.89       458
   macro avg       0.62      0.54      0.57       458
weighted avg       0.88      0.89      0.88       458



## Model 2 - Best model, used in report and test_functions

Same model trained with balanced data

### Data balancing

Data augmentation and oversampling

In [ ]:
import pickle
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical
import cv2
import tensorflow as tf
from imblearn.over_sampling import RandomOverSampler

# load train data
with open(train_data_path, 'rb') as f:
    train_images, train_labels = pickle.load(f)

# resize images, new_size defined in previous cell as (224,224)
train_images_resized = [cv2.resize(image, new_size, interpolation=cv2.INTER_LANCZOS4) for image in train_images]

# convert to numpy arrays
train_images_normalized = np.array(train_images_resized)

# normalize pixels
train_images_normalized = train_images_normalized.astype('float32') / 255.0

# conver to one hot endcoded
train_labels = to_categorical(train_labels, 3)

# split training data into 80% training and 20% validation
train_images, val_images, train_labels, val_labels = train_test_split(train_images_normalized, train_labels, test_size=0.2, random_state=42)

# data augmentation
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# oversampling
# reference: https://imbalanced-learn.org/stable/references/generated/imblearn.over_sampling.RandomOverSampler.html
oversampler = RandomOverSampler(random_state=42)
train_images_reshaped = train_images.reshape(-1, fixed_size[0] * fixed_size[1] * 3)
train_images_resampled, train_labels_resampled = oversampler.fit_resample(train_images_reshaped, train_labels)
train_images_resampled = train_images_resampled.reshape(-1, fixed_size[0], fixed_size[1], 3)

# set batch size
batch_size = 32

# create tensorflow dataset from balanced train data
train_dataset = tf.data.Dataset.from_tensor_slices((train_images_resampled, train_labels_resampled)).shuffle(len(train_images_resampled)).batch(batch_size)

# create tensorflow dataset from validation data
val_dataset = tf.data.Dataset.from_tensor_slices((val_images, val_labels)).batch(batch_size)

# get class distribution to check after oversampling
class_count = np.unique(train_labels_resampled.argmax(axis=1), return_counts=True)
print("Class Count after oversampling:", dict(zip(class_count[0], class_count[1])))


Class Count after oversampling: {0: 1539, 1: 1539, 2: 1539}


### Train model


In [ ]:
train_and_validate(cnn2, train_dataset, val_dataset, epochs)

Epoch 1/250
145/145 [==============================] - 220s 2s/step - loss: 1.2836 - accuracy: 0.8625 - val_loss: 1.0186 - val_accuracy: 0.9332
Epoch 2/250
145/145 [==============================] - 218s 2s/step - loss: 0.3632 - accuracy: 0.9127 - val_loss: 1.4014 - val_accuracy: 0.9081
Epoch 3/250
145/145 [==============================] - 218s 2s/step - loss: 0.5997 - accuracy: 0.9066 - val_loss: 1.9084 - val_accuracy: 0.9332
Epoch 4/250
145/145 [==============================] - 217s 1s/step - loss: 0.1993 - accuracy: 0.9355 - val_loss: 0.7184 - val_accuracy: 0.9207
Epoch 5/250
145/145 [==============================] - 217s 1s/step - loss: 0.1713 - accuracy: 0.9422 - val_loss: 1.2880 - val_accuracy: 0.9186
Epoch 6/250
145/145 [==============================] - 219s 2s/step - loss: 0.1706 - accuracy: 0.9422 - val_loss: 1.4973 - val_accuracy: 0.9332
Epoch 7/250
145/145 [==============================] - 218s 2s/step - loss: 0.1791 - accuracy: 0.9575 - val_loss: 0.6483 - val_accuracy:

In [ ]:
cnn2.save(os.path.join(GOOGLE_DRIVE_PATH, 'Models/custom_cnn3.keras'))

### Test model

In [ ]:
test_loss, test_accuracy = cnn2.evaluate(test_dataset)
print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}")

15/15 [==============================] - 7s 439ms/step - loss: 3.6407 - accuracy: 0.9367
Test Loss: 3.6407, Test Accuracy: 0.9367


In [ ]:
from sklearn.metrics import classification_report
import numpy as np

# make predictions on test dataset
predictions = cnn2.predict(test_dataset)

# convert to class labels
predicted_classes = np.argmax(predictions, axis=1)

# convert the one-hot encoded true labels to class indices
true_labels = []
for _, label in test_dataset:
    true_labels.extend(np.argmax(label.numpy(), axis=1))

# get classification report
report = classification_report(true_labels, predicted_classes)

print(report)

15/15 [==============================] - 6s 398ms/step
              precision    recall  f1-score   support

           0       0.83      0.86      0.85        51
           1       0.96      0.97      0.97       388
           2       0.62      0.53      0.57        19

    accuracy                           0.94       458
   macro avg       0.81      0.79      0.79       458
weighted avg       0.94      0.94      0.94       458



# Model 3

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Dropout, Flatten, Dense, BatchNormalization
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
import numpy as np

# define input shape, takes 224x224 images with 3 channels
input_shape = (224, 224, 3)
input_data = Input(shape=input_shape)

# conv layers
x = Conv2D(32, (3, 3), activation='relu')(input_data)
x = BatchNormalization()(x)
x = MaxPooling2D(pool_size=(2, 2))(x)
x = Dropout(0.2)(x)

x = Conv2D(64, (3, 3), activation='relu')(x)
x = BatchNormalization()(x)
x = MaxPooling2D(pool_size=(2, 2))(x)
x = Dropout(0.25)(x)

x = Conv2D(224, (3, 3), activation='relu')(x)
x = BatchNormalization()(x)
x = MaxPooling2D(pool_size=(2, 2))(x)
x = Dropout(0.2)(x)

# fully connected layers
x = Flatten()(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.2)(x)
output = Dense(3, activation='softmax')(x) # apply softmax for multiclassification

# create model
cnn3 = Model(inputs=input_data, outputs=output)

# compile model with adam optimizer and categorical cross entropy loss function
cnn3.compile(optimizer='adam', loss=CategoricalCrossentropy(), metrics=['accuracy'])

# get model summary
cnn3.summary()


Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 conv2d_9 (Conv2D)           (None, 222, 222, 32)      896       
                                                                 
 batch_normalization_6 (Bat  (None, 222, 222, 32)      128       
 chNormalization)                                                
                                                                 
 max_pooling2d_9 (MaxPoolin  (None, 111, 111, 32)      0         
 g2D)                                                            
                                                                 
 dropout_11 (Dropout)        (None, 111, 111, 32)      0         
                                                                 
 conv2d_10 (Conv2D)          (None, 109, 109, 64)      1849

In [ ]:
train_and_evaluate(cnn3, train_dataset, val_dataset, epochs)

Epoch 1/250
145/145 [==============================] - 272s 2s/step - loss: 8.5308 - accuracy: 0.8646 - val_loss: 259.6351 - val_accuracy: 0.1315
Epoch 2/250
145/145 [==============================] - 267s 2s/step - loss: 1.3936 - accuracy: 0.9467 - val_loss: 67.3537 - val_accuracy: 0.3591
Epoch 3/250
145/145 [==============================] - 267s 2s/step - loss: 0.8741 - accuracy: 0.9621 - val_loss: 21.2086 - val_accuracy: 0.7370
Epoch 4/250
145/145 [==============================] - 267s 2s/step - loss: 0.4971 - accuracy: 0.9673 - val_loss: 9.2624 - val_accuracy: 0.8789
Epoch 5/250
145/145 [==============================] - 268s 2s/step - loss: 0.4628 - accuracy: 0.9766 - val_loss: 11.3479 - val_accuracy: 0.7933
Epoch 6/250
145/145 [==============================] - 267s 2s/step - loss: 0.2680 - accuracy: 0.9818 - val_loss: 4.7791 - val_accuracy: 0.9269
Epoch 7/250
145/145 [==============================] - 267s 2s/step - loss: 0.2523 - accuracy: 0.9842 - val_loss: 2.4848 - val_accu

In [ ]:
cnn3.save(os.path.join(GOOGLE_DRIVE_PATH, 'Models/custom_cnn4.keras'))

In [ ]:
test_loss, test_accuracy = cnn3.evaluate(test_dataset)
print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}")

15/15 [==============================] - 8s 514ms/step - loss: 7.0970 - accuracy: 0.9432
Test Loss: 7.0970, Test Accuracy: 0.9432


In [ ]:
from sklearn.metrics import classification_report
import numpy as np

# make predictions on test dataset
predictions = cnn3.predict(test_dataset)

# convert to class labels
predicted_classes = np.argmax(predictions, axis=1)

# convert the one-hot encoded true labels to class indices
true_labels = []
for _, label in test_dataset:
    true_labels.extend(np.argmax(label.numpy(), axis=1))

# get classification report
report = classification_report(true_labels, predicted_classes)

print(report)

15/15 [==============================] - 7s 450ms/step
              precision    recall  f1-score   support

           0       0.83      0.96      0.89        51
           1       0.98      0.96      0.97       388
           2       0.55      0.58      0.56        19

    accuracy                           0.94       458
   macro avg       0.79      0.83      0.81       458
weighted avg       0.95      0.94      0.94       458

